In [9]:
%pip install timm
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   --- ------------------------------------ 1.6/18.2 MB 10.8 MB/s eta 0:00:02
   --------- ------------------------------ 4.2/18.2 MB 11.3 MB/s eta 0:00:02
   --------------- ------------------------ 6.8/18.2 MB 11.7 MB/s eta 0:00:01
   -------------------- ------------------- 9.4/18.2 MB 11.8 MB/s eta 0:00:01
   --------------------------- ------------ 12.3/18.2 MB 12.4 MB/s eta 0:00:01
   ---------------------------------- ----- 15.7/18.2 MB 13.0 MB/s eta 0:00:01
   ---------------------------------------  18.1/18.2 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 12.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import sys
sys.path.append("src")

from patchcore.patchcore import PatchCore
import patchcore.backbones

c:\Users\otuam\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# 1. Load your images (rectangular, e.g., 256x384)
transform = transforms.Compose([
    transforms.Resize((224,224)),  # Ensure all images are the same size
    transforms.ToTensor()
])

train_dataset = ImageFolder("carpet/train", transform=transform)
test_dataset = ImageFolder("carpet/test", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [3]:
# 2. Configure PatchCore
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PatchCore(device)
backbone = patchcore.backbones.load("resnet50")
layers = ["layer2", "layer3"]  # Use layers appropriate for your backbone

model.load(
    backbone=backbone,
    layers_to_extract_from=layers,
    device=device,
    input_shape=(3, 256, 384),  # Channels, Height, Width
    pretrain_embed_dimension=1024,
    target_embed_dimension=384,
    patchsize=3,
    patchstride=1,
)


c:\Users\otuam\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\otuam\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# 3. Train PatchCore (build memory bank)
model.fit(train_loader)

AttributeError: 'list' object has no attribute 'to'

In [ ]:
# 4. Predict anomalies on test set
scores, masks, labels_gt, masks_gt = model.predict(test_loader)

In [ ]:

# 5. (Optional) Visualize or save results
for i, (score, mask) in enumerate(zip(scores, masks)):
    print(f"Image {i}: Anomaly score = {score}")
    # You can save or display mask here